World Cup Football

You are given three tables, teams, matches and schedules, with the following structures:

create table teams
( country varchar2(3)
, group1 varchar2(1)
, seq_in_group number(1)
);

create table match_schedule
( seq number(1)
, home number(1)
, away number(1)
);

create table match_results
( group1 varchar2(1)
, home number(1)
, away number(1)
, home_goals number(2)
, away_goals number(2)
);

First create the table with the teams and their group assignment (which group and which seat within the group):
 
insert into teams (country, group1, seq_in_group) values ('BRA','A',1);
insert into teams (country, group1, seq_in_group) values ('CRO','A',2);
insert into teams (country, group1, seq_in_group) values ('MEX','A',3);
insert into teams (country, group1, seq_in_group) values ('CMR','A',4);

insert into teams (country, group1, seq_in_group) values (’NLD’,’B’,1);
insert into teams (country, group1, seq_in_group) values (‘ESP’,’B’,2);
insert into teams (country, group1, seq_in_group) values (‘CHL’,’B’,3);
insert into teams (country, group1, seq_in_group) values (‘AUS’,’B’,4);


Next, the table that describes the matches to be played in each group (the sequence of the matches):

insert into match_schedule (seq, home, away) values (1,1,2);
insert into match_schedule (seq, home, away) values (2,3,4);
insert into match_schedule (seq, home, away) values (3,1,3);
insert into match_schedule (seq, home, away) values (4,4,2);
insert into match_schedule (seq, home, away) values (5,4,1);
insert into match_schedule (seq, home, away) values (6,2,3);

Next up is the table MATCH_RESULTS where the outcomes of the matches are recorded:

insert into match_results ( group1 , home, away, home_goals, away_goals) values ('A',1,2,2,1);
insert into match_results ( group1 , home, away, home_goals, away_goals) values ('A',3,4,1,3);
insert into match_results ( group1 , home, away, home_goals, away_goals) values ('A',1,3,0,0);
insert into match_results ( group1 , home, away, home_goals, away_goals) values ('A',4,2,3,1);
 
insert into match_results ( group1 , home, away, home_goals, away_goals) values ('B',1,2,1,1);
insert into match_results ( group1 , home, away, home_goals, away_goals) values ('B',3,4,2,0);
insert into match_results ( group1 , home, away, home_goals, away_goals) values ('B',1,3,3,1);
insert into match_results ( group1 , home, away, home_goals, away_goals) values ('B',4,2,0,3)

After creating a table with all competing teams and assigning them to groups, you need create a query to produce all matches to be played.
seq	group	country	country
1	A	BRA	CRO
2	A	MEX	CMR
3	A	BRA	MEX
4	A	CMR	CRO
5	A	CMR	BRA
6	A	CRO	MEX
1	B	ESP	NDL
2	B	CHL	AUS
3	B	ESP	CHL
4	B	AUS	NDL
5	B	AUS	ESP
6	B	NDL	CHL

Next you need create table to record the match results. 
Group	match	score
A	BRA-CRO	2-1
A	MEX-CMR	1-3
A	BRA-MEX	0-0
A	CMR-CRO	3-1
B	ESP-NDL	1-1
B	CHL-AUS	2-0
B	ESP-CHL	3-1
B	AUS-NDL	0-3

Using this tables, you need construct a query to retrieve the current standings in each group. The result will look like this:
group	country	games   points  goal    goals 
                played		diff    scored                              	
A	CMR	2	6	4	6
A	BRA	2	4	1	2
A	MEX	2	2	-2	1
A	CRO	2	2	-3	2
B	NDL	2	4	3	4
B	ESP	2	4	2	4
B	CHL	2	4	0	3
B	AUS	2	2	-5	0



In [4]:
import pandas as pd
import numpy as np

#######################################################################################################
#########    The goal for this script is to generate 'df_all_matches' , 'df_record_results',     ######
#########'current_standing'  tables from the 'teams', 'match_schedule', 'match_results' tables   ######
#######################################################################################################

# column names of teams, match_schedule and match_results tables
teams_columns = ['country', 'group1', 'seq_in_group']
match_schedule_columns = ['seq','home','away']
match_results_columns = ['group1','home','away','home_goals','away_goals']


# creating 'teams' table
teams=pd.DataFrame(np.array([['BRA','A',1],
                             ['CRO','A',2],
                             ['MEX','A',3],
                             ['CMR','A',4],
                             ['NLD','B',1],
                             ['ESP','B',2],
                             ['CHL','B',3],
                             ['AUS','B',4]
                             ]), columns=teams_columns)

teams[teams_columns[0:-1]] = teams[teams_columns[0:-1]].astype('string')  
teams[teams_columns[-1]] = teams[teams_columns[-1]].astype('int8')


#creating 'match_schedule' table
match_schedule=pd.DataFrame(np.array([[1,1,2],
                                      [2,3,4],
                                      [3,1,3],
                                      [4,4,2],
                                      [5,4,1],
                                      [6,2,3]
                                      ]), columns=match_schedule_columns)
match_schedule[match_schedule_columns] = match_schedule[match_schedule_columns].astype('int8')


#creating 'match_result' table
match_results=pd.DataFrame(np.array([['A',1,2,2,1],
                                     ['A',3,4,1,3],
                                     ['A',1,3,0,0],
                                     ['A',4,2,3,1],
                                     ['B',1,2,1,1],
                                     ['B',3,4,2,0],
                                     ['B',1,3,3,1],
                                     ['B',4,2,0,3]
                                     ]), columns=match_results_columns)

match_results[match_results_columns[0]] = match_results[match_results_columns[0]].astype('string')
match_results[match_results_columns[1:]] = match_results[match_results_columns[1:]].astype('int8')



# merging 'match_schedule' and 'teams' tables to produce all matches to be played, sorting and deleting unnecessary columns
# new table name is 'df_all_matches'

df_all_matches = pd.merge(teams, match_schedule, left_on='seq_in_group', right_on='home', how='left')\
                    .drop(['seq_in_group', 'home'], axis=1)
df_all_matches = pd.merge(df_all_matches, teams, left_on=['away', 'group1'], right_on=['seq_in_group', 'group1'], how='left')\
                    .drop(['seq_in_group', 'away'], axis=1)
df_all_matches = df_all_matches[['seq', 'group1', 'country_x', 'country_y']]\
                                    .sort_values(by=['group1', 'seq'], ignore_index=True)\
                                    .rename(columns={"country_x": "country_home", "country_y": "country_away"})


# merging 'match_results' and 'teams' columns, concatinating 'country_home' with 'country_away
# and 'home_goals' with 'away_goals' columns' to show the match results
# new table name is 'df_record_results'

df_record_results = pd.merge(match_results, teams, left_on=['home', 'group1'], right_on=['seq_in_group', 'group1'], how='left')\
                      .drop(['seq_in_group', 'home'], axis=1)\
                      .rename(columns={"country": "country_home"})
df_record_results = pd.merge(df_record_results, teams, left_on=['away', 'group1'], right_on=['seq_in_group', 'group1'], how='left')\
                      .drop(['seq_in_group', 'away'], axis=1)\
                      .rename(columns={"country": "country_away", "group1": "group"})

df_record_results = df_record_results.astype('str')
df_record_results['match'] = df_record_results['country_home'] + " - " + df_record_results['country_away']
df_record_results['score'] = df_record_results['home_goals'] + " - " + df_record_results['away_goals']
df_record_results = df_record_results.drop(['home_goals', 'away_goals', 'country_home', 'country_away'], axis=1)



# copied 'df_record_results' as 'df_record_results2'
# splited match column between 'country_home' and 'country_away' columns
# splited score column between 'score_home' and 'score_away' columns
# added new column 'delta_home_away' which is delta between 'score_home' and 'score_away' columns

df_record_results2 = df_record_results
df_record_results2[['country_home', 'country_away']] = df_record_results2['match']\
                                                        .apply(lambda x: pd.Series(x.split(' - ')))\
                                                        .rename(columns = {0:'country_home', 1:'country_away'})
df_record_results2[['score_home', 'score_away']] = df_record_results2['score']\
                                                        .apply(lambda x: pd.Series(x.split(' - ')))\
                                                        .rename(columns = {0:'score_home', 1:'score_away'})

df_record_results2[['score_home', 'score_away']] = df_record_results2[['score_home', 'score_away']].astype('int8')

df_record_results2['delta_home_away'] = df_record_results2['score_home'] - df_record_results2['score_away']



# created empty lists 'home_points' and 'away_points' looped through df_record_results2['delta_home_away'] column
# added values to the lists depending whether value is less than or equal or greather than 0

home_points = []
away_points = []
for res in df_record_results2['delta_home_away']:
    if res < 0:
        home_points.append(0)
        away_points.append(3)
    elif res == 0:
        home_points.append(1)
        away_points.append(1)
    else:
        home_points.append(3)
        away_points.append(0)


# added 'home_points' and 'away_points' columns in 'df_record_results2' table and filled with lists values
df_record_results2['home_points'] = pd.Series(home_points, dtype='int8')
df_record_results2['away_points'] = pd.Series(away_points, dtype='int8')


# created 'current_standing_home' and 'current_standing_away' tables to differentiate home and away country parameters
# renamed columns names in both tables while keeping column names for both cases

current_standing_home = df_record_results2[['group','country_home','home_points', 'delta_home_away', 'score_home']]\
                        .rename(columns = {'country_home':'country', 
                                           'home_points':'points', 
                                           'delta_home_away':'goal_diff', 
                                           'score_home':'goals_scored'})

current_standing_away = df_record_results2[['group','country_away','away_points', 'delta_home_away', 'score_away']]\
                        .rename(columns = {'country_away':'country', 
                                           'away_points':'points', 
                                           'delta_home_away':'goal_diff', 
                                           'score_away':'goals_scored'})
current_standing_away['goal_diff'] = current_standing_away['goal_diff'] * (-1)

# appending 'current_standing_home' and 'current_standing_away' tables
# aggrigating summary parameters for each country and calculating  country counts
# creating 'current_standing' table
current_standing = current_standing_home.append(current_standing_away, ignore_index=True)
current_standing['games_played'] = 0

current_standing = current_standing.groupby(['group','country'])\
                    .agg({'games_played':'count', 'points':'sum', 'goal_diff': 'sum', 'goals_scored': 'sum'})\
                    .reset_index()\
                    .sort_values(by=['group', 'points'], ascending=[True, False], ignore_index=True)
